In [2]:
pip install pandas beautifulsoup4 indic-nlp-library


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 7.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving english_train.csv to english_train.csv
Saving english_test.csv to english_test.csv
Saving bengali_train.csv to bengali_train.csv
Saving gujrati_train.csv to gujrati_train.csv
Saving bengali_test.csv to bengali_test.csv
Saving gujrati_test.csv to gujrati_test.csv
Saving hindi_train.csv to hindi_train.csv
Saving hindi_test.csv to hindi_test.csv


In [3]:
import pandas as pd

# Define paths and labels
file_paths = {
    "english_train": "/content/english_train.csv",
    "english_test": "/content/english_test.csv",
    "hindi_train": "/content/hindi_train.csv",
    "hindi_test": "/content/hindi_test.csv",
    "gujarati_train": "/content/gujrati_train.csv",
    "gujarati_test": "/content/gujrati_test.csv",
    "bengali_train": "/content/bengali_train.csv",
    "bengali_test": "/content/bengali_test.csv",
}

datasets = {}

# Load all CSV files
for name, path in file_paths.items():
    try:
        df = pd.read_csv(
            path,
            encoding='utf-8',
            engine='python',
            on_bad_lines='skip'  # <- New syntax for skipping bad lines
        )
        datasets[name] = df
        print(f"✅ Loaded {name}: {df.shape} rows")
    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")



✅ Loaded english_train: (28342, 4) rows
✅ Loaded english_test: (2895, 4) rows
✅ Loaded hindi_train: (21225, 4) rows
✅ Loaded hindi_test: (3000, 4) rows
✅ Loaded gujarati_train: (33630, 4) rows
✅ Loaded gujarati_test: (2999, 4) rows
✅ Loaded bengali_train: (12356, 4) rows
✅ Loaded bengali_test: (2951, 4) rows


In [4]:
!pip install transformers sentencepiece


In [5]:
import pandas as pd

# Define paths and language tags
file_paths = {
    "english_train.csv": "en",
    "english_test.csv": "en",
    "hindi_train.csv": "hi",
    "hindi_test.csv": "hi",
    "gujrati_train.csv": "gu",
    "gujrati_test.csv": "gu",
    "bengali_train.csv": "bn",
    "bengali_test.csv": "bn"
}

# Load and combine
all_data = []

for file_name, lang in file_paths.items():
    df = pd.read_csv(file_name)
    df = df[['Article', 'Summary']].copy()
    df = df.dropna()
    df['lang'] = lang
    all_data.append(df)

data = pd.concat(all_data, ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle the dataset
print(f"✅ Total samples: {len(data)}")
data.head()


✅ Total samples: 107398


,Article,Summary,lang
0,Police charge forward to disperse protesters i...,Two anti-coup protesters were shot dead by rio...,en
1,Budget 2023: 1 ફેબ્રુઆરીએ કેન્દ્રીય નાણાં પ્રધ...,Budget 2023 : ગુરુવારે બજેટ તૈયાર કરતા પહેલા &...,gu
2,सरकारी नौकरी की तलाश कर रहे बेरोजगारों के लिए ...,Eklavya Model Residential Schools Vacancy Deta...,hi
3,नई दिल्ली. कृषि कानून के खिलाफ दिल्ली सीमाओं प...,किसानों की तरफ से ये भी कहा जा रहा है कि जिन ब...,hi
4,#কালনা: ছাড়ি গঙ্গায় নৌকাডুবিতে মৃত্যু দুই পর...,West Bengal News: Body recovered of two touris...,bn


In [6]:
import re

def clean_text(text):
    text = str(text)
    text = re.sub(r'\s+', ' ', text)  # Remove multiple spaces
    text = text.strip()
    return text

data['Article'] = data['Article'].apply(clean_text)
data['Summary'] = data['Summary'].apply(clean_text)


In [7]:
data['lang'].value_counts()


,count
lang,
gu,36629
en,31237
hi,24225
bn,15307


In [8]:
data.to_csv("combined_preprocessed_data.csv", index=False)


In [9]:
!pip install transformers datasets


In [10]:
from transformers import MT5Tokenizer

tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128


In [12]:
def prefix_by_lang(lang):
    return f"summarize in {lang}: "


In [13]:
from torch.utils.data import Dataset

class SummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_input_len, max_target_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        prefix = prefix_by_lang(row['lang'])

        input_text = prefix + row['Article']
        target_text = row['Summary']

        input_enc = tokenizer(
            input_text,
            max_length=self.max_input_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        target_enc = tokenizer(
            target_text,
            max_length=self.max_target_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        labels = target_enc["input_ids"]
        labels[labels == tokenizer.pad_token_id] = -100  # Ignore padding in loss

        return {
            "input_ids": input_enc["input_ids"].squeeze(),
            "attention_mask": input_enc["attention_mask"].squeeze(),
            "labels": labels.squeeze()
        }


In [14]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(data, test_size=0.1, random_state=42)

train_dataset = SummarizationDataset(train_df, tokenizer, MAX_INPUT_LENGTH, MAX_TARGET_LENGTH)
val_dataset = SummarizationDataset(val_df, tokenizer, MAX_INPUT_LENGTH, MAX_TARGET_LENGTH)


In [15]:
from transformers import MT5ForConditionalGeneration

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=068e2c49b334c617dea9e59b439ebe1a33c2412f731c6a01301d5fcc659aa6e0
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [32]:
!pip install rouge_score

!pip install evaluate
import evaluate
import numpy as np # Import numpy to check types

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Filter out negative values (like -100 for padding) before decoding
    decoded_preds = tokenizer.batch_decode([pred[pred >= 0] for pred in predictions], skip_special_tokens=True)
    # Filter out negative values from labels as well, just in case
    decoded_labels = tokenizer.batch_decode([label[label >= 0] for label in labels], skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # --- Start of fix for 'numpy.float64' object has no attribute 'mid' ---
    processed_result = {}
    for key, value in result.items():
        # Check if 'value' is a number or doesn't have the 'mid' attribute
        if isinstance(value, (float, int, np.float32, np.float64)) or not hasattr(value, 'mid'):
             # If it's already a number, use it directly (scaled by 100)
            processed_result[key] = float(value) * 100 if isinstance(value, (float, int, np.float32, np.float64)) else 0.0
        else:
            # Otherwise, it's the expected structured object, use .mid.fmeasure
            processed_result[key] = value.mid.fmeasure * 100
    result = processed_result # Update the result dictionary
    # --- End of fix ---


    prediction_lens = [len(pred.split()) for pred in decoded_preds]
    # Avoid division by zero if decoded_preds is empty
    result["gen_len"] = sum(prediction_lens) / len(prediction_lens) if prediction_lens else 0.0


    return result


In [39]:
#OPTION 2
import numpy as np
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Ensure predictions and labels are numpy arrays
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    # Replace -100 in the labels with pad_token_id for decoding
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)

    # Decode generated summaries and references
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Handle both float and object outputs from ROUGE
    processed_result = {}
    for key, value in result.items():
        if hasattr(value, "mid"):
            processed_result[key] = value.mid.fmeasure * 100
        else:
            processed_result[key] = float(value) * 100

    # Calculate average generated summary length
    prediction_lens = [len(pred.split()) for pred in decoded_preds]
    processed_result["gen_len"] = np.mean(prediction_lens) if prediction_lens else 0.0

    return processed_result


In [33]:
!pip install -U transformers


In [34]:
import transformers
print(transformers.__version__)


4.51.3


In [35]:
from transformers import Seq2SeqTrainingArguments
print(Seq2SeqTrainingArguments)
help(Seq2SeqTrainingArguments)


<class 'transformers.training_args_seq2seq.Seq2SeqTrainingArguments'>
Help on class Seq2SeqTrainingArguments in module transformers.training_args_seq2seq:

class Seq2SeqTrainingArguments(transformers.training_args.TrainingArguments)
 |  Seq2SeqTrainingArguments(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_gra

In [36]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5-multilingual-summarizer",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=3e-5,
    weight_decay=0.01,
    save_total_limit=2,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LENGTH,
    generation_num_beams=4,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    fp16=True,
    report_to="none",  # <--- disables wandb and other external loggers
)



In [37]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


<ipython-input-37-2eb80e718add>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("mt5-multilingual-summarizer")
tokenizer.save_pretrained("mt5-multilingual-summarizer")
